In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://www.marriott.com/hotel-search.mi"
driver.get(url)
time.sleep(1)


In [3]:
findreserve_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/div/section/div[1]/div/div/a')[0]
findreserve_button.click()
time.sleep(1)

In [4]:
brandfilter_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/span[4]')[0]
brandfilter_button.click()
time.sleep(1)

In [5]:
courtyard_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/div/div[2]/div[1]/form/div[2]/div[1]/div/fieldset/div[3]/ul/li[1]/a/span')[0]
courtyard_button.click()
time.sleep(1)

In [6]:
apply_button = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/section[2]/div[2]/div/div/div/div[2]/div[2]/div/div/div/form/button[1]')[0]
apply_button.click()
time.sleep(1)

In [7]:
counter = 1
listurls = []
while counter < 30:
    currenturl = driver.current_url
    print(f'Currently on page: {currenturl}.')
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    
#     nextbutton1
    hotelcards = soup.find('div', {'class' : 'js-property-list-container map-view-scroll-space'})
#     listurls = []
    try:
        for x in hotelcards.find_all('div', {'class' : 'l-row'}):
            try:
                href = x.find('a').attrs['href']

                if 'QuickView' in href:
                    url = {}
                    url['url'] = 'https://www.marriott.com/' + href
                    url['name'] = x.find('a').attrs['title']
                    url['code'] = x.find('a').attrs['data-marsha']
                    listurls.append(url)
        #             print(url)
        #             print()
        #             print('='*50)
        #             print()
            except:
                pass
    except:
        break
    nextbutton = soup.find_all('div', {'role' : 'navigation'})
    nextbutton1 = 'https://www.marriott.com/'
    for x in nextbutton:
        try:
            if counter == 1:
                button = x.find_all('a')[5].attrs['href']
                nextbutton1 = nextbutton1 + button 
            elif counter == 2 or counter == 26:
                button = x.find_all('a')[6].attrs['href']
                nextbutton1 = nextbutton1 + button
            elif counter == 3 or counter == 25:
                button = x.find_all('a')[7].attrs['href']
                nextbutton1 = nextbutton1 + button
            elif counter >= 4 and counter <= 24:
                button = x.find_all('a')[8].attrs['href']
                nextbutton1 = nextbutton1 + button
#             else:
#                 button = x.find_all('a')[5].attrs['href']
#                 nextbutton1 = nextbutton1 + button                
        except:
            continue
    driver.get(nextbutton1)
    counter += 1
listurls_df = pd.DataFrame(listurls)

Currently on page: https://www.marriott.com/search/filterSearch.mi.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=2.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=3.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=4.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=5.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=6.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=7.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=8.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=9.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=10.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=11.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=12.
Currently on page: https://www.marriott.com//search/filterSearch.mi?page=13.
Currently on pag

In [8]:
listurls_df

,url,name,code
0,https://www.marriott.com//search/hotelQuickVie...,Courtyard Aberdeen at Ripken Stadium,BWIBD
1,https://www.marriott.com//search/hotelQuickVie...,Courtyard Abilene Northeast,ABINE
2,https://www.marriott.com//search/hotelQuickVie...,Courtyard Abilene Southwest/Abilene Mall South,ABICY
3,https://www.marriott.com//search/hotelQuickVie...,Courtyard Dallas Addison/Midway,DALAD
4,https://www.marriott.com//search/hotelQuickVie...,Courtyard Dallas Addison/Quorum Drive,DALQD
...,...,...,...
1046,https://www.marriott.com//search/hotelQuickVie...,Courtyard Los Angeles Woodland Hills,LAXCW
1047,https://www.marriott.com//search/hotelQuickVie...,Courtyard Worcester,BOSWR
1048,https://www.marriott.com//search/hotelQuickVie...,Courtyard Yonkers Westchester County,HPNYC
1049,https://www.marriott.com//search/hotelQuickVie...,Courtyard York,HARYK


In [9]:
hotels = []
for index, row in listurls_df.iterrows():
    name = row['name'].replace(' ','-').replace('/','-').lower()
    code = row['code'].lower()
    url = 'https://www.marriott.com/hotels/travel/' + code + '-' + name
    hotel ={}
    hotel['name'] = name
    hotel['url'] = url
    hotels.append(hotel)
hotels_df = pd.DataFrame(hotels)

In [10]:
hotels_df

,name,url
0,courtyard-aberdeen-at-ripken-stadium,https://www.marriott.com/hotels/travel/bwibd-c...
1,courtyard-abilene-northeast,https://www.marriott.com/hotels/travel/abine-c...
2,courtyard-abilene-southwest-abilene-mall-south,https://www.marriott.com/hotels/travel/abicy-c...
3,courtyard-dallas-addison-midway,https://www.marriott.com/hotels/travel/dalad-c...
4,courtyard-dallas-addison-quorum-drive,https://www.marriott.com/hotels/travel/dalqd-c...
...,...,...
1046,courtyard-los-angeles-woodland-hills,https://www.marriott.com/hotels/travel/laxcw-c...
1047,courtyard-worcester,https://www.marriott.com/hotels/travel/boswr-c...
1048,courtyard-yonkers-westchester-county,https://www.marriott.com/hotels/travel/hpnyc-c...
1049,courtyard-york,https://www.marriott.com/hotels/travel/haryk-c...


In [11]:
# nextbutton = soup.find_all('div', {'role' : 'navigation'})
# nextbutton1 = 'https://www.marriott.com/'
# for x in nextbutton:
#     button = x.find_all('a', {'class' : ''}
#     nextbutton1 = nextbutton1 + button
# nextbutton1

In [12]:
# driver.current_url

In [13]:
# driver.get(nextbutton1)

In [14]:
# driver.current_url

In [15]:
# nextbutton = soup.find_all('div', {'role' : 'navigation'})
# nextbutton1 = 'https://www.marriott.com/'
# for x in nextbutton:
#     button = x.find_all('a')[5].attrs['href']
#     nextbutton1 = nextbutton1 + button
# nextbutton1

In [16]:
# hotelcards = soup.find('div', {'class' : 'js-property-list-container map-view-scroll-space'})
# listurls = []
# for x in hotelcards.find_all('div', {'class' : 'l-row'}):
#     try:
#         href = x.find('a').attrs['href']
        
#         if 'QuickView' in href:
#             url = {}
#             url['url'] = 'https://www.marriott.com/' + href
#             url['name'] = x.find('a').attrs['title']
#             url['code'] = x.find('a').attrs['data-marsha']
#             listurls.append(url)
# #             print(url)
# #             print()
# #             print('='*50)
# #             print()
#     except:
#         pass
    

    
# listurls_df = pd.DataFrame(listurls)
# listurls_df

In [17]:
# hotels = []
# for index, row in listurls_df.iterrows():
#     name = row['name'].replace(' ','-').replace('/','-').lower()
#     code = row['code'].lower()
#     url = 'https://www.marriott.com/hotels/travel/' + code + '-' + name
#     hotel['name'] = name
#     hotel['url'] = url
#     hotels.append(url)
#     print(url)

In [20]:
%%time

hotel_list = []
counter = 0
for index, row in hotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Courtyard'
        except:
            hotel['brand'] = 'Courtyard'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df       

{'brand': 'Courtyard', 'name': 'courtyard-aberdeen-at-ripken-stadium', 'url': 'https://www.marriott.com/hotels/travel/bwibd-courtyard-aberdeen-at-ripken-stadium', 'street': '830 Long Drive', 'locality': 'Aberdeen', 'state': 'Maryland', 'postalcode': '21001', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bwibd-courtyard-aberdeen-at-ripken-stadium/?directPageRequest=true', 'coordinate3': '39.532788', 'coordinate4': '-76.190975', 'description': '[ A one-of-a-kind hotel in a one-of-a-kind location, the Courtyard Aberdeen is designed as a replica of Oriole Park at Camden Yards, directly across from RipkenYouth baseball fields as IronBirds stadium.  Our spacious guest rooms offer free Wi-Fi, flexible workspaces, and 40” HDTVs with premium channels. As one of the top hotels in Aberdeen, MD we offer amenities including  the Bistro restaurantt, an indoor pool, 24-hour fitness center, 24/7 Market, and access to walking paths around the baseball fields. Pla

{'brand': 'Courtyard', 'name': 'courtyard-thousand-oaks-agoura-hills', 'url': 'https://www.marriott.com/hotels/travel/oxrtc-courtyard-thousand-oaks-agoura-hills', 'street': '29505 Agoura Road', 'locality': 'Agoura Hills', 'state': 'California', 'postalcode': '91301', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Located on the north side of Malibu Wine Country, the brand new Courtyard by Marriott Thousand Oaks Agoura Hills is your gateway to outdoor adventuring. Explore creeks, waterfalls, and panoramic ocean views from hiking trails in the Santa Monica Mountains. Take the canyon straight to Malibu's beaches, vineyards, event venues, and Pepperdine University. Get down to business at nearby Conejo Valley corporate offices or onsite with our fast and free wifi and collaborative work spaces. Our contemporary ballroom spaces feature high ceilings, mountain views, and a sunset-facing terrace which serve as idyllic backdrops for ex

{'brand': 'Courtyard', 'name': 'courtyard-albion', 'url': 'https://www.marriott.com/hotels/travel/jxnab-courtyard-albion', 'street': '200 South Superior Street', 'locality': 'Albion', 'state': 'Michigan', 'postalcode': '49224', 'coordinate1': '-84.753181', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ The Courtyard by Marriott of Albion, MI is located in the heart of Downtown Albion, just minutes away from Albion College, Spring Arbor University, Hillsdale College, Olivet College, and many others. Within walking distance of our beautiful hotel, you will find several attractions including bars and restaurants, the newly renovated historic Bohm Theatre, a children's museum, a new brewery, and other little shops to visit. If you are travelling on business, you will find Albion's industrial district a few miles from our location. Our hotel is also close to major companies such as Consumer's Energy, Eaton Corporation, Tenneco, Brembo Brakes, and many ot

{'brand': 'Courtyard', 'name': 'courtyard-alexandria-old-town-southwest', 'url': 'https://www.marriott.com/hotels/travel/wasal-courtyard-alexandria-old-town-southwest', 'street': '2700 Eisenhower Avenue', 'locality': 'Alexandria', 'state': 'Virginia', 'postalcode': '22314', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/wasal-courtyard-alexandria-old-town-southwest/?directPageRequest=true', 'coordinate2': '38.80212', 'coordinate3': '-77.079378', 'coordinate4': 'N/A', 'description': '[ Comfort meets convenience at Courtyard Alexandria Old Town/Southwest. Our hotel boasts a prime location just 7.1 miles away from the Ronald Reagan Washington National Airport and near top attractions including the National Science Foundation, Cameron Run Regional Park and Hoffman Town Center. After a full day, retreat to stylish rooms and suites with complimentary Wi-Fi, luxury bedding, ergonomic workstations and flat-panel TVs. During downtime, stay on top of your exercise routine at our fit

{'brand': 'Courtyard', 'name': 'courtyard-altoona', 'url': 'https://www.marriott.com/hotels/travel/aoocy-courtyard-altoona', 'street': '2 Convention Center Drive', 'locality': 'Altoona', 'state': 'Pennsylvania', 'postalcode': '16602', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/aoocy-courtyard-altoona/?directPageRequest=true', 'coordinate3': '40.456279', 'coordinate4': '-78.401788', 'description': "[ Enjoy an exceptional stay at the Courtyard Altoona, meticulously designed for the business and pleasure traveler alike. Relax in our modern lobby, The Bistro, and fitness center. Eat–Drink-Connect in The Bistro, offering delicious breakfast and dinner options as well as cocktails and Starbucks® coffee. (Bistro is currently closed due to Coronavirus Pandemic restrictions.) The Courtyard Altoona offers free Wi-Fi, is conveniently connected to the Blair County Convention Center, and is just minutes from a variety of national and regional restaurants. 

...

{'brand': 'Courtyard', 'name': 'courtyard-wilmington-brandywine', 'url': 'https://www.marriott.com/hotels/travel/ilgbw-courtyard-wilmington-brandywine', 'street': '320 Rocky Run Parkway', 'locality': 'Wilmington', 'state': 'Delaware', 'postalcode': '19803', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/ilgbw-courtyard-wilmington-brandywine/?directPageRequest=true', 'coordinate2': '39.817919', 'coordinate3': '-75.549848', 'coordinate4': 'N/A', 'description': "[ Come on in and enjoy the comfort of our NEWLY RENOVATED LOBBY AND COMMON AREAS!  Whether it's work or leisure that brings you to Delaware, Courtyard Wilmington Brandywine offers the right blend of amenities and services to help you thrive on the road. Wilmington and Brandywine Valley are areas brimming with rich history and natural beauty, and our hotel gives you easy access to experience it all. Rediscover nature at Brandywine Creek State Park, shop around for the best deals at Concord Mall and explore fine art and

{'brand': 'Courtyard', 'name': 'courtyard-hartford-windsor', 'url': 'https://www.marriott.com/hotels/travel/bdlwd-courtyard-hartford-windsor', 'street': '1 Day Hill Road', 'locality': 'Windsor', 'state': 'Connecticut', 'postalcode': '06095 1706', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bdlwd-courtyard-hartford-windsor/?directPageRequest=true', 'coordinate3': '41.872983', 'coordinate4': '-72.666612', 'description': "[ Be your most creative and successful self at Courtyard Hartford Windsor. We are conveniently located between Hartford, CT and Springfield ,MA  and a short ride from Bradley International Airport. Our BDL hotel is located near Travelers, Cigna, VOYA, Hartford Insurance, and the University of Hartford.  Before an eventful day full of sightseeing or meetings, get a great night's sleep in our comfortable accommodations. If you need to stay productive, you can easily do so, using our ergonomic work desks and complimentary Wi-Fi. Sto

{'brand': 'Courtyard', 'name': 'courtyard-boston-woburn-boston-north', 'url': 'https://www.marriott.com/hotels/travel/boscw-courtyard-boston-woburn-boston-north', 'street': '700 Unicorn Park Dr', 'locality': 'Woburn', 'state': 'Massachusetts', 'postalcode': '01801', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/boscw-courtyard-boston-woburn-boston-north/?directPageRequest=true', 'coordinate3': '42.488028', 'coordinate4': '-71.116245', 'description': "[ The contemporary Courtyard Woburn/Boston North Hotel has all the tools you need to thrive. Offering complimentary Wi-Fi, ergonomic desk chairs, well-lit desks, and on-site business services, this Boston hotel is perfect for the frequent business traveler. Our 192 room hotel is located in Woburn, MA, near the intersection of Highways I-93 &amp; I-95. Downtown Boston, Logan International Airport and the best attractions like Fenway Park, Boston Common, Freedom Trail, Boston Public Garden, Boston Nati

{'brand': 'Courtyard', 'name': 'courtyard-los-angeles-woodland-hills', 'url': 'https://www.marriott.com/hotels/travel/laxcw-courtyard-los-angeles-woodland-hills', 'street': '21101 Ventura Boulevard', 'locality': 'Woodland Hills', 'state': 'California', 'postalcode': '91364', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/laxcw-courtyard-los-angeles-woodland-hills/?directPageRequest=true', 'coordinate3': '34.1677284240723', 'coordinate4': '-118.59308624267', 'description': "[ The best location in Woodland Hills.  Located right off the 101 freeway between De Soto and Canoga Avenue in the heart of the San Fernando Valley.  Conveniently situated on Ventura Boulevard next door to Trader Joe's, two blocks from Whole Foods Market and multiple local restaurants within walking distance.  One mile to Warner Center for your business needs and The Village at Westfield Topanga for shopping, dining and people watching.   Self parking $10 per night/car for regis

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Courtyard,courtyard-aberdeen-at-ripken-stadium,https://www.marriott.com/hotels/travel/bwibd-c...,830 Long Drive,Aberdeen,Maryland,21001,Reviews,https://www.marriott.com/hotels/maps/travel/bw...,39.532788,-76.190975,[ A one-of-a-kind hotel in a one-of-a-kind loc...,N/A,4.6,4.3
1,Courtyard,courtyard-abilene-northeast,https://www.marriott.com/hotels/travel/abine-c...,2141 Scottish Road,Abilene,Texas,79601,Reviews,https://www.marriott.com/hotels/maps/travel/ab...,32.472811,-99.689013,[ Enjoy award-winning hospitality at Courtyard...,N/A,4.8,4.8
2,Courtyard,courtyard-abilene-southwest-abilene-mall-south,https://www.marriott.com/hotels/travel/abicy-c...,4350 Ridgemont Drive,Abilene,Texas,79606,https://www.marriott.com/hotels/maps/travel/ab...,32.399075,-99.764522,N/A,[ Our unparalleled services and amenities make...,N/A,4.5,4.4
3,Courtyard,courtyard-dallas-addison-midway,https://www.marriott.com/hotels/travel/dalad-c...,4165 Proton Drive,Addison,Texas,75001,Reviews,https://www.marriott.com/hotels/maps/travel/da...,32.946017,-96.841561,"[ Centrally located in Addison, TX, the newly ...",N/A,4.5,4.6
4,Courtyard,courtyard-dallas-addison-quorum-drive,https://www.marriott.com/hotels/travel/dalqd-c...,15160 Quorum Drive,Addison,Texas,75001,https://www.marriott.com/hotels/maps/travel/da...,32.955263,-96.826087,N/A,[ The Courtyard Addison Quorum Hotel is locate...,N/A,4.6,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,Courtyard,courtyard-los-angeles-woodland-hills,https://www.marriott.com/hotels/travel/laxcw-c...,21101 Ventura Boulevard,Woodland Hills,California,91364,Reviews,https://www.marriott.com/hotels/maps/travel/la...,34.1677284240723,-118.59308624267,[ The best location in Woodland Hills. Locate...,N/A,4.7,4.6
1047,Courtyard,courtyard-worcester,https://www.marriott.com/hotels/travel/boswr-c...,72 Grove Street,Worcester,Massachusetts,01605,https://www.marriott.com/hotels/maps/travel/bo...,42.274855,-71.800588,N/A,[ Feel welcome at Courtyard Worcester. Our inv...,N/A,4.4,4.3
1048,Courtyard,courtyard-yonkers-westchester-county,https://www.marriott.com/hotels/travel/hpnyc-c...,5 Executive Boulevard,Yonkers,New York,10701,https://www.marriott.com/hotels/maps/travel/hp...,40.973031,-73.879569,N/A,[ The Courtyard Yonkers Westchester County is ...,N/A,4.6,4.3
1049,Courtyard,courtyard-york,https://www.marriott.com/hotels/travel/haryk-c...,2799 Concord Road,York,Pennsylvania,17402,Reviews,https://www.marriott.com/hotels/maps/travel/ha...,39.982343,-76.670294,[ The state-of-the-art lobby at Courtyard by M...,N/A,4.5,4.4


In [21]:
hotel_df = hotel_df.drop_duplicates(subset='name')
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Courtyard,courtyard-aberdeen-at-ripken-stadium,https://www.marriott.com/hotels/travel/bwibd-c...,830 Long Drive,Aberdeen,Maryland,21001,Reviews,https://www.marriott.com/hotels/maps/travel/bw...,39.532788,-76.190975,[ A one-of-a-kind hotel in a one-of-a-kind loc...,N/A,4.6,4.3
1,Courtyard,courtyard-abilene-northeast,https://www.marriott.com/hotels/travel/abine-c...,2141 Scottish Road,Abilene,Texas,79601,Reviews,https://www.marriott.com/hotels/maps/travel/ab...,32.472811,-99.689013,[ Enjoy award-winning hospitality at Courtyard...,N/A,4.8,4.8
2,Courtyard,courtyard-abilene-southwest-abilene-mall-south,https://www.marriott.com/hotels/travel/abicy-c...,4350 Ridgemont Drive,Abilene,Texas,79606,https://www.marriott.com/hotels/maps/travel/ab...,32.399075,-99.764522,N/A,[ Our unparalleled services and amenities make...,N/A,4.5,4.4
3,Courtyard,courtyard-dallas-addison-midway,https://www.marriott.com/hotels/travel/dalad-c...,4165 Proton Drive,Addison,Texas,75001,Reviews,https://www.marriott.com/hotels/maps/travel/da...,32.946017,-96.841561,"[ Centrally located in Addison, TX, the newly ...",N/A,4.5,4.6
4,Courtyard,courtyard-dallas-addison-quorum-drive,https://www.marriott.com/hotels/travel/dalqd-c...,15160 Quorum Drive,Addison,Texas,75001,https://www.marriott.com/hotels/maps/travel/da...,32.955263,-96.826087,N/A,[ The Courtyard Addison Quorum Hotel is locate...,N/A,4.6,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,Courtyard,courtyard-los-angeles-woodland-hills,https://www.marriott.com/hotels/travel/laxcw-c...,21101 Ventura Boulevard,Woodland Hills,California,91364,Reviews,https://www.marriott.com/hotels/maps/travel/la...,34.1677284240723,-118.59308624267,[ The best location in Woodland Hills. Locate...,N/A,4.7,4.6
1047,Courtyard,courtyard-worcester,https://www.marriott.com/hotels/travel/boswr-c...,72 Grove Street,Worcester,Massachusetts,01605,https://www.marriott.com/hotels/maps/travel/bo...,42.274855,-71.800588,N/A,[ Feel welcome at Courtyard Worcester. Our inv...,N/A,4.4,4.3
1048,Courtyard,courtyard-yonkers-westchester-county,https://www.marriott.com/hotels/travel/hpnyc-c...,5 Executive Boulevard,Yonkers,New York,10701,https://www.marriott.com/hotels/maps/travel/hp...,40.973031,-73.879569,N/A,[ The Courtyard Yonkers Westchester County is ...,N/A,4.6,4.3
1049,Courtyard,courtyard-york,https://www.marriott.com/hotels/travel/haryk-c...,2799 Concord Road,York,Pennsylvania,17402,Reviews,https://www.marriott.com/hotels/maps/travel/ha...,39.982343,-76.670294,[ The state-of-the-art lobby at Courtyard by M...,N/A,4.5,4.4


In [22]:
hotel_df.to_csv('courtyard_description.csv')